In [1]:
import tensorflow as tf
import os

In [2]:
# get current dir
current_dir=os.path.dirname(os.path.realpath(__name__))

In [3]:
# dataset url
#
#the next command will create a dir called 'content' in your current location - be aware
#

_url='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'

#download command
zip_file=tf.keras.utils.get_file(origin=_url,fname='flower_photos.tgz',extract=True,cache_subdir=current_dir+'/content')

#add dir to path
base_dir=os.path.join(os.path.dirname(zip_file),'flower_photos')

In [4]:
#not needed anymore
del current_dir

In [5]:
_IMAGE_SIZE=224
_BATCH_SIZE=64


In [6]:
# validation_split - split to 20% test & 80% train
data_gen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [7]:
# splitted to training and validation
# 80% train =>

train_gen=data_gen.flow_from_directory(
    base_dir,
target_size=(_IMAGE_SIZE,_IMAGE_SIZE),
batch_size=_BATCH_SIZE,
subset='training'
)

Found 2939 images belonging to 5 classes.


In [8]:
# 20% validation =>

val_gen=data_gen.flow_from_directory(
    base_dir,
target_size=(_IMAGE_SIZE,_IMAGE_SIZE),
batch_size=_BATCH_SIZE,
subset='validation'
)

Found 731 images belonging to 5 classes.


In [9]:
lables='\n'.join(sorted(train_gen.class_indices.keys()))
with open('labels.txt','w') as f:
    f.write(lables)

#del from memory
del lables

In [10]:
_IMG_SHAPE=(_IMAGE_SIZE,_IMAGE_SIZE,3)
base_model=tf.keras.applications.MobileNetV2(
    input_shape=_IMG_SHAPE,
    include_top=False,
    weights='imagenet'
    )

In [11]:
base_model.trainable=False

In [12]:
#short import to make the next command shorter
from tensorflow.keras import layers

# 5 = num of classes
model=tf.keras.Sequential([
    base_model,
layers.Conv2D(32,3,activation='relu'),
layers.Dropout(0.2),
layers.GlobalAveragePooling2D(),
layers.Dense(5,activation='softmax')
])

In [13]:
model.compile(
optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',
metrics=['accuracy']
 )

In [14]:
import PIL

In [15]:
#fitting the model
epochs=10
history=model.fit(
    train_gen,
    epochs=epochs,
    validation_data=val_gen
    )

Epoch 1/10
46/46 [==============================] - 92s 2s/step - loss: 0.9351 - accuracy: 0.7030 - val_loss: 0.5405 - val_accuracy: 0.7880
Epoch 2/10
46/46 [==============================] - 164s 4s/step - loss: 0.3273 - accuracy: 0.8853 - val_loss: 0.4120 - val_accuracy: 0.8618
Epoch 3/10
46/46 [==============================] - 152s 3s/step - loss: 0.1964 - accuracy: 0.9272 - val_loss: 0.4170 - val_accuracy: 0.8495
Epoch 4/10
46/46 [==============================] - 203s 4s/step - loss: 0.1336 - accuracy: 0.9564 - val_loss: 0.4263 - val_accuracy: 0.8659
Epoch 5/10
46/46 [==============================] - 186s 4s/step - loss: 0.0893 - accuracy: 0.9728 - val_loss: 0.4610 - val_accuracy: 0.8577
Epoch 6/10
 3/46 [>.............................] - ETA: 1:34 - loss: 0.0847 - accuracy: 0.9844ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "/Users/kono/projects/ai_learning/flower_classifier/l

TypeError: object of type 'NoneType' has no len()